<a href="https://colab.research.google.com/github/Ja1Pat3L/RadicalX_Challenge2/blob/main/JuniorCoderAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install "shapely<2.0.0"
!pip install langchain
!pip install google-cloud-aiplatform --upgrade
!pip install gradio

In [ ]:
import vertexai
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain.chat_models.vertexai import ChatVertexAI
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda
from langchain.memory import ConversationBufferMemory
import gradio as gr

In [ ]:
from google.colab import auth as google_auth
google_auth.authenticate_user()

In [ ]:
vertexai.init(project="coral-firefly-399618", location="us-central1")

In [ ]:
intent_chain = PromptTemplate.from_template('''You are helping a user solve a coding problem, provided below.

Given the user message, classify its intent as one of the following categories: `asking for idea/strategy`, `asking for answer`, `general coding question`, `other`.

Respond with the topic only and no other words.

<coding_problem>
{coding_problem}
</coding_problem>

<message>
{message}
</message>

Classification''') | ChatVertexAI() | StrOutputParser()

collab_chain = ChatPromptTemplate.from_messages([
    ('system', '''You are working with a user on their Python problem, provided below. They have also written some code. Assume you don't know how to solve the problem.

Please comment on the user message, using emojis where appropriate. If the user talks about their code, please take a look at their code as well. Be vague - don't give any big hints, next steps, solutions, or the answer.

Keep your response 1-2 sentences maximum. Remember - talk like you are a coding partner and not the user's teacher. Don't give any code.

<coding_problem>
{coding_problem}
</coding_problem>

<user_code>
{user_code}
</user_code>

<code_output>
{code_output}
</code_output>'''),
    MessagesPlaceholder(variable_name='chat_history'),
    ('human', '{message}')]) | ChatVertexAI()

code_chain = ChatPromptTemplate.from_messages([
    ('system', '''You are working with a user on their Python problem, provided below. They have also written some code. Assume you don't know how to solve the problem.

Please provide a small snippet of code relevant to the user message. Keep it short and to the point. Don't give any hints, solutions, or the answer.

Use emojis where appropriate. Remember - talk like you are a coding partner and not the user's teacher.

<coding_problem>
{coding_problem}
</coding_problem>

<user_code>
{user_code}
</user_code>'''),
    MessagesPlaceholder(variable_name='chat_history'),
    ('human', '{message}')]) | ChatVertexAI()

default_chain = ChatPromptTemplate.from_messages([
    ('system', '''You are a junior coder helping a user solve a Python coding problem. Reply back to the user's message using plenty of appropriate emojis.

Keep your answer short and to the point. Don't ask follow-up questions.'''),
    MessagesPlaceholder(variable_name='chat_history'),
    ('human', '{message}')]) | ChatVertexAI()

In [ ]:
def response_route(info):
  if 'asking for idea/strategy' in info['intent'].lower():
      #return 'IDEA/STRATEGY\n' + str(collab_chain.invoke(info).content)
      return str(collab_chain.invoke(info).content)
  elif 'asking for answer' in info['intent'].lower():
      return f'''Sounds like you're asking me for the solution. I'm just a junior coder so not sure, but let's figure it out together :)'''
  elif 'general coding question' in info['intent'].lower():
      #return 'GENERAL CODING QUESTION\n' + str(code_chain.invoke(info).content)
      return str(code_chain.invoke(info).content)
  else:
      #return 'DEFAULT\n' + str(default_chain.invoke(info).content)
      return str(default_chain.invoke(info).content)

response_chain = {
    'intent': intent_chain,
    'message': lambda x: x['message'],
    'coding_problem': lambda x: x['coding_problem'],
    'user_code': lambda x: x['user_code'],
    'code_output': lambda x: x['code_output'],
    'chat_history': lambda x: x['chat_history']
} | RunnableLambda(response_route)

In [ ]:
screen_chain = PromptTemplate.from_template('''You are helping a user solve a coding problem, provided below.

Given the user's message and your response, determine if your response meets any of the following criteria:

- Gives the solution to the coding problem
- Gives away the entire strategy for solving the coding problem
- Code that contains the entire solution to the coding problem
- Providing starter code that gives away the entire solution

Small talk is fine. Respond with a single character, either 'Y' for yes or 'N' for no.

<coding_problem>
{coding_problem}
</coding_problem>

<message>
{message}
</message>

<response>
{response}
</response>

Classification''') | ChatVertexAI() | StrOutputParser()

def output_route(info):
  if 'n' in info['screen'].lower():
      return info['response']
  elif 'y' in info['screen'].lower():
      #debug_msg = 'SCREENED OUT\n' + info['response'] + '\n'
      debug_msg = ''
      return debug_msg + f'''I'm sorry, I don't have a good response to that. You can say it in a different way or tell me something else :)'''
  else:
      return 'There was an error with the output screening'

output_chain = {
    'screen': screen_chain,
    'message': lambda x: x['message'],
    'coding_problem': lambda x: x['coding_problem'],
    'response': lambda x: x['response'],
} | RunnableLambda(output_route)

In [ ]:
global coding_problem

coding_problem = f'''
Description:
Write a Python function that takes a list of integers as input and returns the sum of all even numbers in the list. Please store your answer in a variable 'ans'.

Inputs:
nums = [5, 10, 15, 20, 25, 30]
'''

memory = ConversationBufferMemory(return_messages=True)
memory.save_context({'input': f'''Let's practice coding in Python.'''}, {'output': f'''Let's do it! Here's a problem that I found:\n{coding_problem}'''})

with gr.Blocks() as demo:

  def get_completion(msg, user_code, code_output, script):
    global coding_problem

    response_inputs = {'coding_problem': coding_problem, 'message': msg, 'user_code': user_code, 'code_output': code_output, 'chat_history': memory.load_memory_variables({})['history']}
    response = response_chain.invoke(response_inputs)
    output = output_chain.invoke({'coding_problem': coding_problem, 'message': msg, 'response': response})

    memory.save_context({'input': msg}, {'output': output})
    script.append((msg, output))

    return '', script

  def run_code(code):
    try:
        local_vars = {}
        exec(code, None, local_vars)

        out_string = 'Your code ran successfully!\n'
        for var, val in local_vars.items():
            out_string += f'''\n{var} = {val}'''

        return out_string

    except Exception as e:
        return 'ERROR\n' + str(e)

  with gr.Row():
    with gr.Column():
      chatbot = gr.Chatbot([(f'''Let's practice coding in Python.''', f'''Let's do it! Here's a problem that I found:\n{coding_problem}''')])
      chat_input = gr.Textbox(value='', label='Chat with junior coder')

    with gr.Column():
      code = gr.Code()
      code_submit = gr.Button(value='Run')
      code_output = gr.Textbox(value='', label='Code Output')

  chat_input.submit(get_completion, [chat_input, code, code_output, chatbot], [chat_input, chatbot])
  code_submit.click(run_code, [code], [code_output])

demo.launch()


Thanks for being a Gradio user! If you have questions or feedback, please join our Discord server and chat with us: https://discord.gg/feTf9x3ZSB
Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://7bbe17255ba30031bd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
